In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [2]:
#filename = 'edges karate.csv'
filename = 'edges lesmis.csv'
edges = pd.read_csv(filename, sep=';', header=None, names=['id1', 'id2', 'weight'])
edges.head()

,id1,id2,weight
0,1,0,1
1,2,0,8
2,3,0,10
3,3,2,6
4,4,0,1


In [3]:
edges.head()

,id1,id2,weight
0,1,0,1
1,2,0,8
2,3,0,10
3,3,2,6
4,4,0,1


In [4]:
G = nx.from_pandas_edgelist(edges, 'id1', 'id2', edge_attr=True)
G = G.to_directed() # E[1, 2] and E[2, 1] must be different references

In [5]:
G.nodes()

NodeView((1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 13, 14, 15, 17, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76))

In [6]:
G.edges()

OutEdgeView([(1, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 11), (2, 0), (2, 3), (2, 11), (3, 0), (3, 2), (3, 11), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (11, 10), (11, 3), (11, 2), (11, 0), (11, 12), (11, 13), (11, 14), (11, 15), (11, 23), (11, 24), (11, 25), (11, 26), (11, 27), (11, 28), (11, 29), (11, 31), (11, 32), (11, 33), (11, 34), (11, 35), (11, 36), (11, 37), (11, 38), (11, 43), (11, 44), (11, 48), (11, 49), (11, 51), (11, 55), (11, 58), (11, 64), (11, 68), (11, 69), (11, 70), (11, 71), (11, 72), (10, 11), (12, 11), (12, 23), (13, 11), (14, 11), (15, 11), (17, 16), (17, 18), (17, 19), (17, 20), (17, 21), (17, 22), (17, 23), (16, 17), (16, 18), (16, 19), (16, 20), (16, 21), (16, 22), (16, 23), (16, 26), (16, 55), (18, 16), (18, 17), (18, 19), (18, 20), (18, 21), (18, 22), (18, 23), (19, 16), (19, 17), (19, 18), (19, 20), (19, 21), (19, 22), (19, 23), (20, 16), (20, 17), (20, 18), (20, 19), (20, 21), (20, 22), (20, 23), (21, 16), (21

In [7]:
def get_weight(G, u, v):
    return G.get_edge_data(u, v)['weight']

def dependency_coefficient(G, x, v, y):
    return get_weight(G, v, y) / (get_weight(G, x, v) + get_weight(G, v, y))

for x, y in G.edges():
    edge = G.get_edge_data(x, y)
    w = edge['weight']
    N = set(G.neighbors(x))
    #CN = nx.common_neighbors(G, x, y) # not implemented for directed type
    CN = N & set(G.neighbors(y))
    
    edge['D'] = (w + sum(get_weight(G, x, v) * dependency_coefficient(G, x, v, y) for v in CN)) / sum(get_weight(G, x, v) for v in N)

In [8]:
#filename_verification = 'edges_dep_directed karate.csv'
filename_verification = 'edges_dep_directed lesmis.csv'
edges_verification = pd.read_csv(filename_verification, sep=';', header=None, names=['id1', 'id2', 'dep verification'])
edges_verification.head()

,id1,id2,dep verification
0,1,0,1.000000
1,0,1,0.032258
2,2,0,0.801471
3,0,2,0.439516
4,3,0,0.805451


In [9]:
edges_verification['dep'] = edges_verification.apply(lambda row: G.get_edge_data(row['id1'], row['id2'])['D'], axis=1)
edges_verification.head()

,id1,id2,dep verification,dep
0,1,0,1.000000,1.000000
1,0,1,0.032258,0.032258
2,2,0,0.801471,0.801471
3,0,2,0.439516,0.439516
4,3,0,0.805451,0.805451


In [10]:
np.testing.assert_allclose(edges_verification['dep'], edges_verification['dep verification'])

In [11]:
edges_verification.to_csv('verification' + filename_verification)